In [ ]:
# save a 1h product for ICOS cities portal
df_1_h_despiked = df_1_m_spike_detected.sort("creation_timestamp") \
        .filter(pl.col("Flag") == 'O') \
        .drop("Flag", "OriginalFlag") \
        .group_by_dynamic("creation_timestamp", every='1h', by=["system_id", "sys_name_short"]) \
        .agg(pl.all().exclude(["creation_timestamp","sys_name_short"]).mean(),
             pl.col("gmp343_corrected").std().alias("Stdev"),
             pl.col("gmp343_corrected").count().alias("NbPoints")) \
        .with_columns(pl.when(pl.col("NbPoints") < 40).then(pl.lit(389)).otherwise(pl.lit(0)).alias("OriginalFlag")) \
        .with_columns(
                pl.when(pl.col("OriginalFlag") > 0).then(pl.lit('K')).otherwise(pl.lit('O')).alias("Flag"),
                (pl.col("creation_timestamp") + timedelta(minutes=30)))
             
df_1_h_despiked.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "pipeline", "flagged_1_h_acropolis.parquet"))